In [2]:
library(Matrix)
library(xgboost)
library("caret")

# Download data

In [1]:
########### download raw data
### Download 10x snATAC-seq raw data from NCBI Gene Expression Omnibus (GSE169453)
### Download 10x multiome raw data from NCBI Gene Expression Omnibus (GSE200044)

########### Download processed snATAC-seq and multiome data to reproduce figures
### There several ways to download processed data.
###1, download files less than 25M from github data folder: https://github.com/gaoweiwang/Islet_snATACseq
### or, from figshare:
### multiome: https://figshare.com/articles/dataset/processed_multiome_zip/19497665
### snATACseq: https://figshare.com/articles/dataset/processed_snATACseq_zip/19497656

###2, download large processed or intermeidate files:
#snATAC-seq data (http://169.228.232.194/~mmallick/o/processed_snATACseq.tar.gz)
#multiome data (http://169.228.232.194/~mmallick/o/processed_multiome.tar.gz)

########### change working directory
wd_snATAC = '/oasis/tscc/scratch/gaw006/snATACseq/processed/'
wd_multiome = '/oasis/tscc/scratch/gaw006/multiome/processed/'

# subcluster

In [1]:
rm(list = ls())
wd_snATAC = '/oasis/tscc/scratch/gaw006/snATACseq/processed/'

# read in matrix data using the Matrix package
indata <- Matrix::readMM(paste0(wd_snATAC,'snATAC_500bp.mtx'))  
indata@x[indata@x > 0] <- 1
dim(indata)

# format cell info
cellinfo <- read.table(paste0(wd_snATAC,'snATAC_500bp.barcodes'))
cell_ID <- as.character(cellinfo$V1)

# format peak info
peakinfo <- read.table(paste0(wd_snATAC,'snATAC_500bp.regions'))
peak_ID <- as.character(peakinfo$V1)

rownames(indata)=cell_ID
colnames(indata)=peak_ID

indata[1:4,1:4]

[1] 218973 404661

4 x 4 sparse Matrix of class "dgTMatrix"
                        1:100000097-100000597 1:100001226-100001726
JYH792_AAACGAAAGCATTGGG                     .                     .
JYH792_AAACGAAAGGGTCTGA                     .                     .
JYH792_AAACGAAAGTAGACCG                     .                     .
JYH792_AAACGAACAAATAGTG                     .                     .
                        1:100009946-100010446 1:100014513-100015013
JYH792_AAACGAAAGCATTGGG                     .                     .
JYH792_AAACGAAAGGGTCTGA                     .                     .
JYH792_AAACGAAAGTAGACCG                     .                     .
JYH792_AAACGAACAAATAGTG                     .                     .

In [4]:
keep_gender=!(grepl("X", peak_ID) | grepl("Y", peak_ID))
indata=indata[,keep_gender]
dim(indata)
length(peak_ID[(grepl("X", peak_ID) | grepl("Y", peak_ID))])

[1] 218973 395657

[1] 9004

In [5]:
#### barcode annotation
M=read.csv(paste0(wd_snATAC,'barcode_info.csv'))

M_donor=as.character(M$donor)
M_leiden=as.numeric(M$leiden)

C=read.csv(paste0(wd_snATAC,'meta_data.csv'))
###subset
C=C[(as.character(C$donor) %in% M_donor),]

donor_all=as.character(C$donor)
disease_all=as.character(C$Sample.Description.Name)
donor_ND=donor_all[disease_all=='Non']
donor_PD=donor_all[disease_all=='Pre']
donor_T2D=donor_all[disease_all=='T2D']

M_disease=M_donor
M_disease[M_donor %in% donor_ND]='ND'
M_disease[M_donor %in% donor_PD]='PD'
M_disease[M_donor %in% donor_T2D]='T2D'

M_celltype=M_donor
alpha_ID=c(0,4,8)
beta_ID=c(1,2,3,5)
delta_ID=c(6)
gamma_ID=c(10)
acinar_ID=c(7)
ductal_ID=c(9)
stellate_ID=c(11)
endothelial_ID=c(13)
immune_ID=c(12)
M_celltype[M_leiden %in% alpha_ID]='alpha'
M_celltype[M_leiden %in% beta_ID]='beta'
M_celltype[M_leiden %in% delta_ID]='delta'
M_celltype[M_leiden %in% gamma_ID]='gamma'
M_celltype[M_leiden %in% acinar_ID]='acinar'
M_celltype[M_leiden %in% ductal_ID]='ductal'
M_celltype[M_leiden %in% stellate_ID]='stellate'
M_celltype[M_leiden %in% endothelial_ID]='endothelial'
M_celltype[M_leiden %in% immune_ID]='immune'

M$celltype=M_celltype
M$disease=M_disease
M[1:3,]

index,donor,duplicated_reads,frac_duplicated_reads,frac_mito_reads,frac_promoters_used,frac_reads_in_peaks,frac_reads_in_promoters,log10_n_counts,log10_n_peaks,⋯,reads_in_promoters,total_sequenced_reads,tss_used,unique_mito_reads,unique_usable_reads,leiden,n_count_500bp,n_peak_500bp,celltype,disease
JYH792_AAACGAAAGCATTGGG,JYH792,1805,0.1117855,0.0066071554,0.09164685,0.4976297,0.2753131,4.158000,3.832892,⋯,3891,16147,1773,94,14133,4,7277,3541,alpha,ND
JYH792_AAACGAAAGGGTCTGA,JYH792,1904,0.1234040,0.0007437709,0.06606017,0.4494976,0.2059546,4.133858,3.802979,⋯,2767,15429,1278,10,13435,3,6125,3008,beta,ND
JYH792_AAACGAAAGTAGACCG,JYH792,2062,0.1829474,0.0297566372,0.03613150,0.3530954,0.1631513,3.944828,3.632963,⋯,1431,11271,699,269,8771,4,3309,1712,alpha,ND


In [6]:
indata=indata[(rownames(indata) %in% as.character(M$index)),]
all(as.character(M$index)==rownames(indata))
dim(indata)

########################################
######### using beta cell as example
keep_cell=(M$celltype=='beta')
P=read.csv(paste0(wd_snATAC,'peak_celltype_500_all.csv'))
P_all=as.character(P$X)
P_beta=P_all[as.numeric(P$beta)>0]
length(P_beta)
keep_peak=(colnames(indata) %in% P_beta)

data_use=indata[keep_cell,keep_peak]
M_use=M[keep_cell,]
dim(data_use)
all(rownames(data_use)==as.character(M_use$index))

[1] TRUE

[1] 218973 395657

[1] 230573

[1]  92780 224671

[1] TRUE

In [7]:
row_sum=Matrix::rowSums(data_use)
col_sum=Matrix::colSums(data_use)
min(row_sum)
min(col_sum)

[1] 98

[1] 58

In [8]:
## remove cells with <1000 reads and peaks counted in <100 cells
keep_cell=(row_sum>1000)
keep_peak=(col_sum>100)

data_use=data_use[keep_cell,keep_peak]
M_use=M_use[keep_cell,]
dim(data_use)
all(rownames(data_use)==as.character(M_use$index))

[1]  90290 224573

[1] TRUE

## training and testing of each donor

In [11]:
M_new=read.csv(paste0(wd_snATAC,'AI_beta_subtype.csv'))
all(as.character(M_use$index)==as.character(M_new$index))

beta_subtype = as.character(M_new$subtype)
disease_enrich = beta_subtype
disease_enrich[beta_subtype == 'beta_1'] ='ND'
disease_enrich[beta_subtype == 'beta_2'] ='T2D'

M_new$pre_disease = disease_enrich

M_new[1:3,]

[1] TRUE

X.1,X,index,donor,celltype,disease,subtype,subtype_probability,pre_disease
1,1,JYH792_AAACGAAAGGGTCTGA,JYH792,beta,ND,beta_2,0.9991231561,T2D
2,2,JYH792_AAACGAAGTACAAATG,JYH792,beta,ND,beta_1,0.0002105963,ND
3,3,JYH792_AAACGAAGTTCCCAAA,JYH792,beta,ND,beta_1,0.0024496897,ND


## HPAP

In [12]:
## test data

H_matrix <- Matrix::readMM(paste0(wd_snATAC,'HPAP_beta_500.mtx')) 
H_matrix@x[H_matrix@x > 0] <- 1
dim(H_matrix)

# format cell info
cellinfo <- read.table(paste0(wd_snATAC,'HPAP_beta_500.barcodes'))
cell_ID <- as.character(cellinfo$V1)

# format peak info
peakinfo <- read.table(paste0(wd_snATAC,'HPAP_beta_500.regions'))
peak_ID <- as.character(peakinfo$V1)

rownames(H_matrix)=cell_ID
colnames(H_matrix)=peak_ID

keep_gender=!(grepl("X", peak_ID) | grepl("Y", peak_ID))
H_matrix=H_matrix[,keep_gender]
length(peak_ID[(grepl("X", peak_ID) | grepl("Y", peak_ID))])


P=read.csv(paste0(wd_snATAC,'peak_celltype_500_all.csv'))
P_all=as.character(P$X)
P_beta=P_all[as.numeric(P$beta)>0]
length(P_beta)
keep_peak=(colnames(H_matrix) %in% P_beta)
H_use=H_matrix[,keep_peak]
dim(H_use)

Meta=read.csv(paste0(wd_snATAC,'HPAP_beta_barcode.csv'))
all(rownames(H_use)==as.character(Meta$X))
row_sum=rowSums(H_use)
col_sum=colSums(H_use)
keep_cell=(row_sum>1000)
keep_peak=(colnames(H_use) %in% colnames(data_use))

H_use=H_use[keep_cell,keep_peak]
Meta=Meta[keep_cell,]
all(rownames(H_use)==as.character(Meta$X))
dim(H_use)

name_test=colnames(H_use)
H_use=H_use[,order(name_test)]
all(colnames(H_use)==colnames(data_use))

[1]  11588 412113

[1] 9840

[1] 230573

[1]  11588 225524

[1] TRUE

[1] TRUE

[1]  11261 224573

[1] TRUE

In [13]:
dim(Meta)
dim(H_use)

[1] 11261    26

[1]  11261 224573

In [13]:
temp_label=rep('no',dim(Meta)[1])

keep_2=(as.character(M_new$disease)!='PD')
keep_3=(as.character(M_new$disease)==as.character(M_new$pre_disease))
keep_train=(keep_2 & keep_3)
train.x = data_use[keep_train,]
dim(train.x)

train_disease=as.character(M_new$disease)
train_d=rep(0,length(train_disease))
train_d[train_disease=='T2D']=1   #######
train_d[train_disease=='ND']=0  #######
train.y = train_d[keep_train]
length(train.y)

train.x1=as(train.x, "dgCMatrix")
bst <- xgboost(data = train.x1, label = train.y, max.depth = 60, eta = 0.2, nthread = 24, nrounds = 80, objective = "binary:logistic")
    

[1]  48568 224573

[1] 48568

[1]	train-error:0.001977 
[2]	train-error:0.001400 
[3]	train-error:0.001153 
[4]	train-error:0.000700 
[5]	train-error:0.000721 
[6]	train-error:0.000535 
[7]	train-error:0.000371 
[8]	train-error:0.000350 
[9]	train-error:0.000329 
[10]	train-error:0.000247 
[11]	train-error:0.000206 
[12]	train-error:0.000206 
[13]	train-error:0.000185 
[14]	train-error:0.000165 
[15]	train-error:0.000165 
[16]	train-error:0.000144 
[17]	train-error:0.000124 
[18]	train-error:0.000124 
[19]	train-error:0.000082 
[20]	train-error:0.000082 
[21]	train-error:0.000082 
[22]	train-error:0.000041 
[23]	train-error:0.000021 
[24]	train-error:0.000021 
[25]	train-error:0.000021 
[26]	train-error:0.000021 
[27]	train-error:0.000021 
[28]	train-error:0.000021 
[29]	train-error:0.000021 
[30]	train-error:0.000000 
[31]	train-error:0.000000 
[32]	train-error:0.000000 
[33]	train-error:0.000000 
[34]	train-error:0.000000 
[35]	train-error:0.000000 
[36]	train-error:0.000000 
[37]	train-error:0.000000 
[38]	train

In [14]:
## or load the model
load(paste0(wd_snATAC,'bst.Rdata'))

In [15]:
test.x=H_use
test.x1=as(test.x, "dgCMatrix")
pred <- predict(bst, test.x1)

pred_label=as.numeric(pred > 0.5)
pred_label1=pred_label
pred_label1[pred_label==0]='beta_1' #######
pred_label1[pred_label==1]='beta_2' #######

Meta$subtype_probability=pred
Meta$subtype=pred_label1 #######
Meta[1:10,]

X.1,X,unique_usable_reads,total_sequenced_reads,duplicated_reads,unique_mito_reads,reads_in_peaks,reads_in_promoters,tss_used,frac_reads_in_peaks,⋯,n_counts,batch,donor,norm_log_counts,log10_usable_counts,log_usable_counts,leiden,cluster,subtype_probability,subtype
6,HPAP035_AAACGAATCACACGTA,101548,215711,111268,2217,66863,25074,7252,0.6584374,⋯,103307,0,HPAP035,7.173991,4.129754,9.509110,0,0,0.0012818115,beta_1
9,HPAP035_AAACGAATCTTAAGCG,207243,460996,252758,45,129281,52646,10171,0.6238136,⋯,211815,0,HPAP035,7.138429,4.426137,10.191557,0,0,0.0152109042,beta_1
11,HPAP035_AAACTCGGTATATGGA,47859,108112,59805,199,32913,14041,4875,0.6877076,⋯,48663,0,HPAP035,7.258825,3.839667,8.841159,0,0,0.0003714848,beta_1
23,HPAP035_AAAGATGTCGGTGATT,57050,138030,79660,1067,38053,11811,4443,0.6670114,⋯,58153,0,HPAP035,7.073844,3.836704,8.834337,0,0,0.9864307642,beta_2
30,HPAP035_AAAGGGCAGAAGAGTG,60013,153377,91237,1824,41381,15088,5355,0.6895339,⋯,60884,0,HPAP035,7.230316,3.924589,9.036701,0,0,0.0003594154,beta_1
32,HPAP035_AAAGGGCCAAAGAAGG,125610,287446,158946,2217,79159,29096,8186,0.6301966,⋯,128584,0,HPAP035,7.062808,4.176525,9.616805,0,0,0.0059291855,beta_1
33,HPAP035_AAAGGGCCAATGGCAG,67178,234741,164585,2663,47712,15186,5119,0.7102325,⋯,68397,0,HPAP035,7.166784,3.947532,9.089528,0,0,0.0004194723,beta_1
34,HPAP035_AAAGGGCCATCATAGC,70666,161143,89526,547,47345,17025,5647,0.6699827,⋯,72134,0,HPAP035,7.108043,3.945124,9.083983,0,0,0.0002662277,beta_1
35,HPAP035_AAAGGGCCATCGTGAT,35879,87290,51054,191,23926,9109,3624,0.6668525,⋯,36505,0,HPAP035,7.175447,3.678609,8.470311,0,0,0.0005264148,beta_1
40,HPAP035_AAATGAGCACGTTAGT,92395,205943,112160,938,57737,19885,6525,0.6248931,⋯,94021,0,HPAP035,7.104812,4.058806,9.345745,0,0,0.8699650168,beta_2


In [15]:
# save results
#write.csv(Meta,paste0(wd_snATAC,'AI_HPAP_beta.csv')) #######
